In [1]:
from dotenv import load_dotenv 
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.messages.messages_request_builder import MessagesRequestBuilder
from msgraph.generated.search.query.query_post_request_body import QueryPostRequestBody
from msgraph.generated.search.query.query_request_builder import QueryRequestBuilder
from msgraph.generated.search.search_request_builder import SearchRequestBuilder
from msgraph.generated.models.search_request import SearchRequest
from msgraph.generated.models.entity_type import EntityType
from msgraph.generated.models.search_query import SearchQuery
from msgraph.generated.models.search_response import SearchResponse
import os
import json
import asyncio 
import uuid

In [2]:
from typing import Tuple

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding

In [3]:
kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
api_key, org_id = sk.openai_settings_from_dot_env()
#kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-4", api_key, org_id))
kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10b210110>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10eedadd0>}

In [4]:
#print(org_id)

In [5]:
load_dotenv()

True

In [6]:
TENANT_ID = os.getenv("TENANT_ID")

In [7]:
CLIENT_ID = os.getenv("CLIENT_ID")

In [8]:
graphUserScopes = os.getenv("graphUserScopes").split(' ')

In [9]:
#print(CLIENT_ID)

In [10]:
gabby = '9c72876c-b0ec-41b3-8079-8ebe216daa34'

In [11]:
#fabs = 'fabian@fabster.onmicrosoft.com'

In [12]:
## ensure that you have turned on DefaultClientType to Yes in the App Registration

In [13]:
print(graphUserScopes)

['User.Read', 'Mail.ReadWrite', 'Chat.ReadWrite', 'Calendars.Read', 'Files.Read.All', 'Sites.Read.All']


In [14]:
credential = DeviceCodeCredential(client_id=CLIENT_ID, tenant_id=TENANT_ID) 
scopes = graphUserScopes
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [16]:
result = await client.me.get()

In [17]:
print(result.display_name)

Gabby Gabgab


## Work with Teams(Chat) Search Query Endpoint

In [18]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.ChatMessage
			],
			query = SearchQuery(
				query_string = "GabbyG",
			),
		),
	],
)

chat_result = await client.search.query.post(request_body)

In [19]:
chat_data = chat_result.value

In [20]:
##Testing with GUID
search_response = chat_data[0]  # Assuming there's only one SearchResponse object in the list

# Initialize an empty list to store summaries, chat IDs, and generated GUIDs
summaries_chat_ids_and_guids = []

# Iterate over hits_containers in the SearchResponse object
for hits_container in search_response.hits_containers:
    # Iterate over each hit in the container
    for hit in hits_container.hits:
        # Extract the summary
        summary = hit.summary

        # Extract the chatId from additional_data
        chat_id = hit.resource.additional_data.get('chatId', 'Unknown')

        # Generate a unique GUID for each hit
        guid = str(uuid.uuid4())

        # Append the chatId, summary, and GUID as a tuple to the list
        summaries_chat_ids_and_guids.append((guid, summary))

# Now 'summaries_chat_ids_and_guids' contains the chatId, summary, and a unique GUID for each search hit
for item in summaries_chat_ids_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=summary)
    print(item)

                                            

('fb1c1e64-a163-4e09-ac23-d89c98885406', 'I prefer dinner in thanksgiving dinner amsterdam this year over thanksgiving dinner in sweden last year...')
('79b89d78-14dc-496e-972e-d5cdb791862d', "I can't get over the thanksgiving dinner we had in amsterdam.")
('2a465f3a-3670-4117-8f0e-5f0bc90d3b42', '"id": "9c72876c-b0ec-41b3-8079-8ebe216daa34"')
('4e094870-5e90-4205-a792-9bede7e8d7c8', 'Fabian Williams  lets practice the demo while we are in Antwerp  sounds like a plan')
('b4b54850-9e1f-4491-a937-4d36c5e1b593', "Fabian Williams  hello Gabby, are you prepared for the ChatGPT session with Python?  yes! i'm very excited.")
('ff7295ff-2b8b-4a5a-90b0-6d0a546b633d', 'lets practice the demo while we are in Antwerp...')
('bc1073e4-b701-4de8-99ca-b35de5290ede', 'hello Gabby, are you prepared for the ChatGPT session with Python?')


In [21]:
#print(summaries_chat_ids_and_guids)

## Work with Drives(Files) Search Query Endpoint

In [24]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.DriveItem
			],
			query = SearchQuery(
				query_string = "GabbyG",
			),
		),
	],
)

drive_result = await client.search.query.post(request_body)

In [25]:
drive_data = drive_result.value
#print(drive_data)

In [28]:
try:
    drive_search_response = drive_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    drive_summary_from_and_guids = []
    drive_narrative = []

    if drive_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for drive_hits_container in drive_search_response.hits_containers:
            # Iterate over each hit in the container
            for drive_hit in drive_hits_container.hits:
                # Extract the summary
                drive_summary = drive_hit.summary

                # Extract the 'from' information from additional_data
                from_data = drive_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')
                file_name = getattr(drive_hit.resource, 'name', 'Unknown')
                user_email = from_data.get('email', 'Unknown')

                # Generate a unique GUID for each hit
                drive_guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                drive_summary_from_and_guids.append((drive_guid, drive_summary, from_name, from_email))
                drive_narrative.append((drive_guid, file_name))
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'drive_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in drive_narrative:
    await kernel.memory.save_information_async("ESPC23", id=drive_guid, text=file_name)
    print(item)



('c4b21d35-5271-41b8-a91d-f372c152f9aa', 'CollabSummit2024_GabbySpeaking.docx')
('809dfb5f-c1db-45b4-9607-829c555cbad6', 'Apps')


In [29]:
print(drive_narrative)

[('c4b21d35-5271-41b8-a91d-f372c152f9aa', 'CollabSummit2024_GabbySpeaking.docx'), ('809dfb5f-c1db-45b4-9607-829c555cbad6', 'Apps')]


## Work with Messages(Email) Search Query Endpoint

In [30]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.Message
			],
			query = SearchQuery(
				query_string = "GabbyG",
			),
		),
	],
)

mail_result = await client.search.query.post(request_body)

In [31]:
mail_data = mail_result.value

In [32]:
try:
    mail_search_response = mail_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    mail_summary_from_and_guids = []

    if mail_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for mail_hits_container in mail_search_response.hits_containers:
            # Iterate over each hit in the container
            for mail_hit in mail_hits_container.hits:
                # Extract the summary
                mail_summary = mail_hit.summary

                # Extract the 'from' information from additional_data
                from_data = mail_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')

                # Generate a unique GUID for each hit
                guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                mail_summary_from_and_guids.append((guid, mail_summary, from_name, from_email))
                mailconstruct = mail_summary + " from " + from_name
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'mail_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in mail_summary_from_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=mailconstruct)
    print(item)

('17a7a64c-743f-4d4b-8f4b-8b124cde08c6', "Fabian Williams shared a file with you  Here's the document that Fabian Williams shared with you.   ProjectRoundtreeStatusReport   This link will work for anyone.  Open   Privacy...", 'Unknown', 'Unknown')
('798fb252-1e72-493b-86f4-63bf01e83768', "Fabian Williams shared a file with you   Here's the document that Fabian Williams shared with you.   HowToGainAccessToStudyGroup  1 min to read this.   This link will work for...", 'Unknown', 'Unknown')
('8f691a53-0324-4b49-8add-d3233dd5ecee', 'From: Microsoft <microsoft-noreply@microsoft.com> Sent: Thursday, November 9, 2023 8:15 PM To: Fabian Williams <fabian@fabster.onmicrosoft.com> Subject: Your Microsoft invoice...', 'Unknown', 'Unknown')
('a2b19147-ae6b-46ad-a9b4-8c4b4db2264e', '...email is intended for <c0>GabbyG</c0>@fabster.onmicrosoft.com For people in Canada This is a mandatory service communication. To set your contact preferences for other communications, visit...', 'Unknown', 'Unknown')


In [ ]:
print(mail_summary_from_and_guids)

## Query and Read All 3 Payloads from Semantic Memory ONLY

In [49]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "Whats going on with Gabby",
        "What conferences are we doing",
    ]

    for question in questions:
        results = await kernel.memory.search_async("ESPC23", question, limit=3)
        for result in results:
            print(f"Question: {question}")
            print(f"Answer: {result.text}\nRelevance: {result.relevance}\n")

In [50]:
await search_memory_examples(kernel)

Question: Whats going on with Gabby
Answer: hello Gabby, are you prepared for the ChatGPT session with Python?
Relevance: 0.8014901065953683

Question: Whats going on with Gabby
Answer: Apps
Relevance: 0.7280218217552771

Question: Whats going on with Gabby
Answer: Ok, So make sure you know your lines, study your answers to the questions. Get your demo ready to go. MOST IMPORTANT: do all the sacrifices to the demo gods!!! Dad. from Unknown
Relevance: 0.7222264229726226

Question: What conferences are we doing
Answer: Apps
Relevance: 0.748952142093939

Question: What conferences are we doing
Answer: hello Gabby, are you prepared for the ChatGPT session with Python?
Relevance: 0.7329083577823825

Question: What conferences are we doing
Answer: Ok, So make sure you know your lines, study your answers to the questions. Get your demo ready to go. MOST IMPORTANT: do all the sacrifices to the demo gods!!! Dad. from Unknown
Relevance: 0.7305096495158402

